<a href="https://www.kaggle.com/code/johnycoder/rsna-inference?scriptVersionId=162623185" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np, pandas as pd, tensorflow as tf, os, pydicom as dicom, cv2

TEST_IMG_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images'
IMG_SIZE = [256, 256]
BATCH_SIZE = 16
ORIGINAL_COLS = [
        "bowel_healthy", "bowel_injury", "extravasation_healthy", "extravasation_injury",
        "kidney_healthy", "kidney_low", "kidney_high",
        "liver_healthy", "liver_low", "liver_high",
        "spleen_healthy", "spleen_low", "spleen_high",
    ]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
model = tf.keras.models.load_model('/kaggle/input/rsna-train/rsna-atd.keras')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 8, 8, 2048)   23587712    ['input_1[0][0]']                
                                                                                                  
 global_average_pooling2d (Glob  (None, 2048)        0           ['resnet50[0][0]']               
 alAveragePooling2D)                                                                              
                                                                                              

In [3]:
def post_proc(pred):
    proc_pred = np.empty((len(pred[0]), 2*2 + 3*3), dtype="float32")

    # bowel, extravasation
    proc_pred[:,0] = np.transpose(np.array(pred[:][0]))
    proc_pred[:, 1] = 1 - proc_pred[:, 0]
    proc_pred[:, 2] = np.transpose(np.array(pred[:][1]))
    proc_pred[:, 3] = 1 - proc_pred[:, 2]
    
    # liver, kidney, sneel
    proc_pred[:, 4:7] = pred[:][2]
    proc_pred[:, 7:10] = pred[:][3]
    proc_pred[:, 10:13] = pred[:][4]

    return proc_pred

test_ims = []
patient_ids = os.listdir(TEST_IMG_PATH)
for patient_id in patient_ids:
    directory = os.listdir(os.path.join(TEST_IMG_PATH, patient_id))[0] #taking only the first one, dont know what the second does yet
    file = os.listdir(os.path.join(TEST_IMG_PATH, patient_id, directory))[0]
    file_path = os.path.join(TEST_IMG_PATH, patient_id, directory, file)
    #print(file_path)
    #print('/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images/63706/39279/30.dcm')
    ds = dicom.dcmread(file_path)
    res = cv2.resize(ds.pixel_array, dsize=IMG_SIZE, interpolation=cv2.INTER_CUBIC)
    res = np.stack([res]*3,axis=-1)
    test_ims.append([res])
test_dataset = tf.data.Dataset.from_tensor_slices((test_ims))
test_dataset.batch(BATCH_SIZE)
#predictions = model.predict(test_dataset)
predictions = pd.DataFrame(post_proc(model.predict(test_dataset)))
predictions.columns = ORIGINAL_COLS
#predictions.set_index([pd.Index(patient_ids), 'patient_id'])
# Set the named indices as the index of the DataFrame
#predictions = predictions.rename_axis(index={i: name for i, name in enumerate(patient_ids)})
predictions['patient_id'] = patient_ids
#predictions.set_index('patient_id')
# Align with sample submission
sub_df = pd.read_csv(f"/kaggle/input/rsna-2023-abdominal-trauma-detection/sample_submission.csv")
sub_df = sub_df[["patient_id"]]
sub_df['patient_id'] = sub_df['patient_id'].astype(np.int64)
predictions['patient_id'] = predictions['patient_id'].astype(np.int64)
sub_df = sub_df.merge(predictions, on="patient_id", how="left")
display(sub_df)
    
# Store submission
sub_df.to_csv("submission.csv",index=False)
sub_df.head(2)

3/3 [==============================] - 2s 10ms/step


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
0,48843,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,9.468534e-12,2.163769e-28,0.713664,0.123721,0.162614
1,50046,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,5.796696e-11,2.517921e-26,0.713664,0.123721,0.162614
2,63706,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.545234e-11,7.827782e-28,0.713664,0.123721,0.162614


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
0,48843,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,9.468534e-12,2.163769e-28,0.713664,0.123721,0.162614
1,50046,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,5.796696e-11,2.517921e-26,0.713664,0.123721,0.162614


# Credits 

https://www.kaggle.com/code/metric/rsna-trauma-metric/notebook

https://keras.io/examples/keras_recipes/tfrecord/

https://www.kaggle.com/code/amyjang/tensorflow-pneumonia-classification-on-x-rays

https://www.kaggle.com/code/aritrag/kerascv-starter-notebook-infer